In [1]:
%load_ext autoreload
%autoreload 0

import numpy as np
from multiprocessing import Process, Queue, Pool
import os
from stupid_tails import run_experiment
from markov import markov_chain, markov_iter_rng, markov_iter

In [2]:
def run_experiment_multi(n, M):

    exp = dict()

    exp["M"] = M
    exp["n"] = n

    parse = set("")

    tail_symbols = ""

    for k in range(1, n + 1):
        seq = ""
        m_iter = markov_iter(M)

        while seq in parse:
            seq += next(m_iter)

        parse.add(seq)
        tail_symbols += next(m_iter)

        # print("\tThis tree was {} characters long,\n\twith tail symbol {}".format(len(seq), tail_symbols[-1]))

        exp[str(k)] = seq

    exp["tail_symbols"] = tail_symbols
    
    return exp



def run_simu(n, M, N):
    return [run_experiment_multi(n, M) for _ in range(N)]

In [11]:
%%time

M = markov_chain(2)
n = 2000
n_exp = 500

exps = [run_experiment(n, M) for _ in range(n_exp)]

CPU times: user 2min 1s, sys: 47.7 s, total: 2min 49s
Wall time: 1min 58s


In [7]:
%%time

M = markov_chain(2)
n = 2000
n_exp = 500
jobs = []
q = Queue()

d = 7
N = n_exp // d
pool = Pool()
r = sum([pool.apply_async(run_simu, (n, M, N)).get() for _ in range(d)], [])

TypeError: markov_iter() missing 1 required positional argument: 'rng'

In [6]:
def rng_gen(N, n):
    rands = np.random.rand(N*n*(n-1)//2)
    for r in rands:
        yield r

def run_simu_mr(M, n, N):
    
    rng = rng_gen(N, n)
    
    exps = []
    
    for _ in range(N):
        exp = dict()
        exp["M"] = M
        exp["n"] = n
        
        tail_symbols = ""
        parse = set("")
        
        for k in range(1, n+1):
            seq = ""
            m_iter = markov_iter_rng(M, rng)
            
            while seq in parse:
                seq += next(m_iter)
            
            parse.add(seq)
            tail_symbols += next(m_iter)
   
            exp[str(k)] = seq
    
        exp["tail_symbols"] = tail_symbols
        exps.append(exp)
    
    return exps

def run_simu(n, M, N):
    return [run_experiment_multi(n, M) for _ in range(N)]

In [7]:
%%time

M = markov_chain(2)
n = 2000
n_exp = 500
q = Queue()

d = 7
N = n_exp // d
pool = Pool()
r = sum([pool.apply_async(run_simu_mr, (M, n, N)).get() for _ in range(d)], [])

CPU times: user 256 ms, sys: 77.6 ms, total: 334 ms
Wall time: 24.7 s
